In [128]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
from sklearn.metrics import roc_auc_score  
from sklearn.model_selection import KFold,StratifiedKFold,RepeatedStratifiedKFold      
from bayes_opt import BayesianOptimization    
from functools import partial               
import lightgbm as lgb                      
import warnings
from catboost import CatBoostClassifier
warnings.filterwarnings("ignore") 

pd.set_option('display.max_columns', 1500)

In [129]:
# datetime 함수

def make_datetime(x):
    # string 타입의 Time column을 datetime 타입으로 변경
    x     = str(x)
    year  = int(x[:4])
    month = int(x[4:6])
    day   = int(x[6:8])
    hour  = int(x[8:10])
    #mim  = int(x[10:12])
    #sec  = int(x[12:])
    return dt.datetime(year, month, day, hour)

In [130]:
# train_data load
train_qual = pd.read_csv('./data/train_quality_data.csv')
train_err = pd.read_csv('./data/train_err_data.csv')
train_problem = pd.read_csv('./data/train_problem_data.csv')
time_tr=pd.read_csv('./data/train_time_count.csv')


In [131]:
fwver_tr = pd.read_csv('./data/fwver_train_most_recent.csv')
fwver_te = pd.read_csv('./data/fwver_test_most_recent.csv')
problem_train_fwver = pd.read_csv('./data/problem_train_fwver_count.csv')
problem_test_fwver = pd.read_csv('./data/problem_test_fwver_count.csv')

In [132]:
# test_data load
test_err = pd.read_csv('./data/test_err_data.csv')
test_qual = pd.read_csv('./data/test_quality_data.csv')
test_submission = pd.read_csv('./data/sample_submission.csv')
time_te=pd.read_csv('./data/test_time_count.csv')


In [133]:
# datetime 변환 후 주말 1, 평일 0 으로 저장

train_err['datetime'] = train_err['time'].apply(make_datetime)
test_err['datetime'] = test_err['time'].apply(make_datetime)
train_err['dayname'] = pd.to_datetime(train_err['datetime'], format='%Y%m%d').map(lambda x : x.strftime('%A'))
d1 = {'Monday':0, 'Tuesday':0, 'Wednesday':0, 'Thursday':0,
      'Friday':0, 'Saturday':1, 'Sunday':1}
train_err['we_or_wk'] = train_err['dayname'].map(d1)

test_err['dayname'] = pd.to_datetime(test_err['datetime'], format='%Y%m%d').map(lambda x : x.strftime('%A'))
d1 = {'Monday':0, 'Tuesday':0, 'Wednesday':0, 'Thursday':0,
      'Friday':0, 'Saturday':1, 'Sunday':1}

test_err['we_or_wk'] = test_err['dayname'].map(d1)

In [134]:
# 11월 동안의 데이터만 사용 

train_err = train_err.loc[(train_err.datetime >= pd.to_datetime('2020-11-01 00:00:00')) & (train_err.datetime <= pd.to_datetime('2020-11-30 23:59:59'))]
test_err = test_err.loc[(test_err.datetime >= pd.to_datetime('2020-11-01 00:00:00')) & (test_err.datetime <= pd.to_datetime('2020-11-30 23:59:59'))]

In [135]:
# train, test error 데이터에 주, 일, 시간 데이터 분리

train_err['week'] = train_err.datetime.dt.isocalendar().week
test_err['week'] = test_err.datetime.dt.isocalendar().week

train_err['day'] = train_err.datetime.dt.day
test_err['day'] = test_err.datetime.dt.day

train_err['hour'] = train_err.datetime.dt.hour
test_err['hour'] = test_err.datetime.dt.hour

# 파생변수 생성
+ 유저별 최신 펌웨어 버전 (fwver_most_recent)
+ 유저별 시간대별 (시간대를 4 섹션으로 구분) error 발생 횟수 카운트 (time_count)
+ 유저별 가장 오래 사용한 펌웨어 버전 (longest_used_fwver)
+ 유저별 펌웨어 버전 변화하는 것 잡아내는 것 (fwver_change)
+ 유저별 가장 많이 사용한 펌웨어 버전 (most_used_fwver)
+ train_problem에 있는 펌웨어 버전에 해당하는지 (problem_fwver_check)
+ train_problem에 있는 펌웨어 버전에 몇 번 해당하는지 (problem_fwver_check_count)
+ 유저별 connection error 카운트 (connection_user_count)
+ err 데이터에서 유저별 요일별 error 발생 횟수 카운트 (day_count)
+ 유저별 펌웨어 버전 변경 횟수 카운트 (fw_changed_count)
+ 요일별 errtype의 평균, 표준편차, 최댓값 (day_errtype)
+ 유저 - 모델명 사용 여부만 (user_model_count1)
+ 유저 - 모델명 err에 찍힌 개수만큼 (user_model_count2)
+ 유저 - 모델명 err에 찍힌 개수만큼 (연월일 기준 중복 제거) (user_model_count3)
+ 주말/주중, 주별, 일별, 시간별 에러코드의 평균, 표준편차, 최댓값

+ model_nm별 카운트
+ err_type별 카운트
+ err_code 상위 50개 카운트
+ quality issue 유무
+ connect 관련 error 발생 카운트
+ 전체 err_type 합계

## fwver_most_recent

In [136]:
def fwver_most_recent(data):
    data.sort_values(by = ['user_id', 'time'], inplace = True)
    data2 = data[['user_id', 'time', 'fwver']]
    data2.drop_duplicates('user_id', keep = 'last', inplace = True)
    data2['fwver'] = data2['fwver'].apply(lambda x: x[:5])
    data2['fwver'] = data2['fwver'].apply(lambda x: x if x != '8.5.3' else '8.5')
    fwver_most_recent = data2[['user_id', 'fwver']].reset_index(drop = True)
    fwver_most_recent['fwver'] = fwver_most_recent['fwver'].str.lstrip('0')

    return fwver_most_recent

## time_count

In [137]:
def time_count1(x):
    if x >= '06' and x <= '11':
        return 1
    else:
        return 0

def time_count2(x):
    if x >= '12' and x <= '17':
        return 1
    else:
        return 0
    
def time_count3(x):
    if x >= '18' and x <= '23':
        return 1
    else:
        return 0

def time_count4(x):
    if x >= '00' and x <= '05':
        return 1
    else:
        return 0

In [138]:
def time_count(data):
    data['time'] = data['time'].apply(lambda x: str(x)[8:10])
    
    data['time_06_11'] = data['time'].apply(time_count1)
    data['time_12_17'] = data['time'].apply(time_count2)
    data['time_18_23'] = data['time'].apply(time_count3)
    data['time_00_05'] = data['time'].apply(time_count4)
    
    time_count = data.groupby('user_id')['time_06_11', 'time_12_17', 'time_18_23', 'time_00_05'].sum()
    time_count = time_count.reset_index()
    
    return time_count

## longest_used_fwver

In [139]:
def make_datetime2(x):
    # string 타입의 Time column을 datetime 타입으로 변경
    x     = str(x)
    year  = int(x[:4])
    month = int(x[4:6])
    day   = int(x[6:8])
    
    return dt.datetime(year, month, day)

In [140]:
def most_used_fwver(data):
    data['datetime2'] = data['time'].apply(make_datetime2)
    data = pd.DataFrame(data.groupby(['user_id', 'datetime2'])['fwver'].value_counts())
    data.columns = ['fwver_count']
    data.reset_index(inplace = True)
    data = pd.DataFrame(data.groupby(['user_id', 'fwver'])['fwver_count'].sum())
    data = data.sort_values(by = 'fwver_count', ascending = False)
    data.reset_index(inplace = True)
    data = data.drop_duplicates('user_id', keep = 'first')
    data = data.sort_values('user_id', ascending = True)
    data['count'] = 1
    data = pd.pivot_table(data, index = 'user_id', columns = 'fwver', values = 'count', fill_value = 0).reset_index()
    data_col = ['fwv_' + str(i) for i in data.columns[1:]]
    data_col.insert(0, 'user_id')
    data.columns = data_col
    return data

## fwver_change

In [141]:
# fwver 변화하는 것 잡아내는 함수

def get_fwver_change(data):
    data['datetime2'] = data['time'].apply(make_datetime2)
    data = pd.DataFrame(data.groupby(['user_id', 'datetime2'])['fwver'].value_counts())
    data.columns = ['fwver_count']
    data.reset_index(inplace = True)
    data = pd.DataFrame(data.groupby(['user_id', 'fwver'])['fwver_count'].sum())
    data = data.reset_index()
    
    users = data['user_id'].unique()
    tmp = []
    
    for user in users:
        tmp.append(tuple(data[data['user_id'] == user]['fwver'].unique()))
        
    fw = list(set(tmp))
    return fw

In [142]:
tr_fw = get_fwver_change(train_err)
te_fw = get_fwver_change(test_err)

In [143]:
# train, test의 fwver 변화한 것 모두 합침

fw_change_list = list(set(tr_fw + te_fw))
fw_change_list

[('04.33.1185',),
 ('04.16.3439',),
 ('04.22.1666',),
 ('04.22.1778',),
 ('05.15.2114', '05.15.2138'),
 ('04.22.1750', '04.22.1778'),
 ('04.16.3553', '04.16.3571', '04.33.1261'),
 ('04.16.3553', '04.33.1125', '04.33.1261'),
 ('04.33.1261', '05.66.3237', '05.66.3571'),
 ('04.22.1684', '04.22.1750', '04.22.1778', '05.15.2138'),
 ('04.16.3553', '04.16.3571', '04.33.1149'),
 ('04.22.1448',),
 ('04.16.3553', '04.16.3571', '04.33.1125', '04.33.1261'),
 ('04.22.1750', '04.22.1778', '05.15.2138'),
 ('05.15.2138',),
 ('04.22.1608', '04.22.1750', '04.22.1778'),
 ('04.22.1750', '04.22.1778', '04.33.1261'),
 ('04.73.2237',),
 ('04.33.1125', '04.33.1261', '05.15.2138'),
 ('04.16.3553', '04.22.1750', '04.22.1778'),
 ('04.16.3569', '04.16.3571'),
 ('03.11.1167', '04.33.1149', '04.33.1261'),
 ('03.11.1149',),
 ('04.73.2237', '04.73.2571'),
 ('04.82.1730', '04.82.1778'),
 ('04.33.1261', '05.66.3237'),
 ('03.11.1167', '04.16.3553', '04.33.1261'),
 ('04.22.1750',),
 ('03.11.1167', '04.33.1185', '04.33.12

In [144]:
# fwver 변화 잡는거 이어서 데이터프레임 만들기

def make_fwver_change(data, train=True):
    data['datetime2'] = data['time'].apply(make_datetime2)
    data = pd.DataFrame(data.groupby(['user_id', 'datetime2'])['fwver'].value_counts())
    data.columns = ['fwver_count']
    data.reset_index(inplace = True)
    data = pd.DataFrame(data.groupby(['user_id', 'fwver'])['fwver_count'].sum())
    data = data.reset_index()
    
    users = data['user_id'].unique()
    
    fwver_change = pd.DataFrame()
    if train == True:
        fwver_change['user_id'] = range(10000, 25000)
    else:
        fwver_change['user_id'] = range(30000, 44999)
    
    for col in fw_change_list:
        column = '_'.join(col)
        fwver_change[column] = 0
        
    for user in users:
        col_name = tuple(data[data['user_id'] == user]['fwver'].unique())
        col_name = '_'.join(col_name)
        fwver_change.loc[fwver_change['user_id'] == user, col_name] = 1
    
    return fwver_change

## most_used_fwver

In [145]:
def most_used_fwver(data, train = True):
    data['datetime2'] = data['time'].apply(make_datetime2)
    data = pd.DataFrame(data.groupby(['user_id', 'datetime2'])['fwver'].value_counts())
    data.columns = ['fwver_count']
    data.reset_index(inplace = True)
    data = pd.DataFrame(data.groupby(['user_id', 'fwver'])['fwver_count'].sum())
    data = data.sort_values(by = 'fwver_count', ascending = False)
    data.reset_index(inplace = True)
    data = data.drop_duplicates('user_id', keep = 'first')
    data = data.sort_values('user_id', ascending = True)
    data['count'] = 1
    users = data['user_id'].unique()
    most_used = pd.DataFrame()
    if train == True:
        most_used['user_id'] = range(10000, 25000)
    else:
        most_used['user_id'] = range(30000, 44999)
    
    for col in fw_list:
        column = 'fwv_' + col
        most_used[column] = 0
    
    for user in users:
        col_name = data[data['user_id'] == user]['fwver']
        col_name = 'fwv_' + col_name
        most_used.loc[most_used['user_id'] == user, col_name] = 1
    
    return most_used    

## problem_fwver_check

In [146]:
def problem_fwver_check(data, train = True):
    
    users = data['user_id'].unique()
    
    problem_fwver_check = pd.DataFrame()
    
    if train == True:
        problem_fwver_check['user_id'] = range(10000, 25000)
    else:
        problem_fwver_check['user_id'] = range(30000, 44999)
    
    for col in train_problem_fw_list:
        problem_fwver_check[col] = 0
    
    for user in users:
        for col in train_problem_fw_list:
            if col in data[data['user_id'] == user]['fwver'].tolist():
                problem_fwver_check.loc[problem_fwver_check['user_id'] == user, col] = 1
    
    return problem_fwver_check

## problem_fwver_check_count

In [147]:
def problem_fwver_check_count(data):
    data = problem_fwver_check_count(data)
    data['problem_fw_count'] = data[data.columns[1:]].sum(axis = 1)
    data2 = data[['user_id', 'problem_fw_count']]
    
    return data2

## connection_err_count

In [148]:
def connection_err_count(data):
    data = data[['user_id', 'time', 'errcode']]
    data['count'] = 1
    data2 = data[(data['errcode'] == 'connection timeout') | (data['errcode'] == 'connection fail to establish') | 
                 (data['errcode'] == 'connectionterminated by local host') | (data['errcode'] == 'connection fail for LMP response timout') | 
                 (data['errcode'] == 'L2CAP connection cancelled')]
    data3 = pd.pivot_table(data2, index = 'user_id', columns = 'errcode', values = 'count', aggfunc = 'sum', fill_value = 0).reset_index()
    data4 = data[['user_id']].drop_duplicates().reset_index(drop = True)
    data5 = pd.merge(data4, data3, how = 'left')
    data5 = data5.fillna(0)
    return data5

## day_count

In [149]:
def user_dayname_count(data):
    data = pd.DataFrame(data.groupby('user_id')['dayname'].value_counts())
    data.columns = ['day_count']
    data.reset_index(inplace = True)
    data = pd.pivot_table(data, values = 'day_count', index = 'user_id', columns = 'dayname', fill_value = 0).reset_index()
    data = data[['user_id', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']]
    return data

## day_errtype

In [150]:
def day_errtype(data, num_of_users, first_index):

    data['day'] = data.datetime.dt.isocalendar().day

    df3 = data[['user_id', 'errtype', 'day']]

    df4 = df3[['user_id', 'day', 'errtype']].value_counts().to_frame().reset_index()

    df5 = df4.sort_values(['user_id', 'day']).rename(columns = {0 : 'counts'}).reset_index(drop = True)

    tmp1 = df5.loc[df5['day'] == 1][['user_id', 'errtype', 'counts']].values
    tmp2 = df5.loc[df5['day'] == 2][['user_id', 'errtype', 'counts']].values
    tmp3 = df5.loc[df5['day'] == 3][['user_id', 'errtype', 'counts']].values
    tmp4 = df5.loc[df5['day'] == 4][['user_id', 'errtype', 'counts']].values
    tmp5 = df5.loc[df5['day'] == 5][['user_id', 'errtype', 'counts']].values
    tmp6 = df5.loc[df5['day'] == 6][['user_id', 'errtype', 'counts']].values
    tmp7 = df5.loc[df5['day'] == 7][['user_id', 'errtype', 'counts']].values
    
    day_data = np.zeros((num_of_users, 42, 7))

    for i, dfa in enumerate([tmp1, tmp2, tmp3, tmp4, tmp5, tmp6, tmp7]):
        for inx, val1, val2 in tqdm(dfa):
            day_data[:, :, i][inx - first_index, val1 - 1] = val2

    day_data_mean = day_data.mean(axis = 2)
    day_data_std = day_data.std(axis = 2)
    day_data_max = day_data.max(axis = 2)

    ddmean = pd.DataFrame(day_data_mean, columns = ['err_type_day_mean' + str(i) for i in range(1, 43)])
    ddstd = pd.DataFrame(day_data_std, columns = ['err_type_day_std' + str(i) for i in range(1, 43)])
    ddmax = pd.DataFrame(day_data_max, columns = ['err_type_day_max' + str(i) for i in range(1, 43)])
    
    ddmean.drop('err_type_day_mean29', axis = 1, inplace = True)
    ddstd.drop('err_type_day_std29', axis = 1, inplace = True)
    ddmax.drop('err_type_day_max29', axis = 1, inplace = True)
    
    return ddmean, ddstd, ddmax

In [151]:
train_errtype_day_mean, train_errtype_day_std, train_errtype_day_max = day_errtype(train_err, 15000, 10000)

100%|██████████████████████████████████████████████████████████████████████| 156326/156326 [00:00<00:00, 366082.56it/s]


In [152]:
test_errtype_day_mean, test_errtype_day_std, test_errtype_day_max = day_errtype(test_err, 14999, 30000)

100%|██████████████████████████████████████████████████████████████████████| 157413/157413 [00:00<00:00, 355432.54it/s]


## user_model_count1 (유저 - 모델명 사용 여부만)

In [153]:
def user_model_count1(data):
    data = data[['user_id', 'time', 'model_nm']]
    data['count'] = 1
    data = pd.pivot_table(data, index = 'user_id', columns = 'model_nm', values = 'count', fill_value = 0).reset_index()
    return data

## user_model_count2 (유저 - 모델명 err에 찍힌 개수만큼)

In [154]:
def user_model_count2(data):
    data = data[['user_id', 'time', 'model_nm']]
    data['count'] = 1
    data['time'] = data['time'].apply(lambda x: str(x)[:8])
    data = pd.DataFrame(data.groupby('user_id')['model_nm'].value_counts())
    data.columns = ['model_count']
    data.reset_index(inplace = True)
    data = pd.pivot_table(data, index = 'user_id', columns = 'model_nm', values = 'model_count', fill_value = 0).reset_index()
    return data

## user_model_count3 (유저 - 모델명 err에 찍힌 개수만큼 (연월일 기준 중복 제거))

In [155]:
def user_model_count3(data):
    data = data[['user_id', 'time', 'model_nm']]
    data['count'] = 1
    data['time'] = data['time'].apply(lambda x: str(x)[:8])
    data = data.drop_duplicates(['user_id', 'time', 'model_nm'], keep = 'last')
    data = pd.DataFrame(data.groupby('user_id')['model_nm'].value_counts())
    data.columns = ['model_count']
    data.reset_index(inplace = True)
    data = pd.pivot_table(data, index = 'user_id', columns = 'model_nm', values = 'model_count', fill_value = 0).reset_index()
    return data

## 주말/주중, 주별, 일별, 시간별 에러코드의 평균, 표준편차, 최댓값

In [156]:
def we_or_wk(df,num_df_user,first_index):
    
#     df = df.loc[(df.datetime >= pd.to_datetime('2020-11-01 00:00:00')) & (df.datetime <= pd.to_datetime('2020-11-30 23:59:59'))]
    datas = df[['user_id','errtype','we_or_wk']]
    df_ = datas[['user_id','we_or_wk','errtype']].value_counts().to_frame().reset_index()
    df_ = df_.sort_values(['user_id','we_or_wk']).rename(columns = {0:'counts'}).reset_index(drop = True)

    df1 = df_.loc[df_.we_or_wk == 0][['user_id','errtype','counts']].values
    df2 = df_.loc[df_.we_or_wk == 1][['user_id','errtype','counts']].values

    day_data = np.zeros((num_df_user,42,2))
    for i, dfa in enumerate([df1,df2]):
        for inx, val1,val2 in dfa:
            day_data[:, :, i][inx-first_index,val1 - 1] = val2

    m = day_data.mean(axis = 2)
    std = day_data.std(axis = 2)
    m_2 = day_data.max(axis = 2)

    m = pd.DataFrame(m, columns = ['err_type_weorwk_mean' + str(i) for i in range(1, 43)])
    std = pd.DataFrame(std, columns = ['err_type_weorwk_std' + str(i) for i in range(1, 43)])
    m_2 = pd.DataFrame(m_2, columns = ['err_type_weorwk_max' + str(i) for i in range(1, 43)])

    m.drop('err_type_weorwk_mean29', axis = 1, inplace = True)
    std.drop('err_type_weorwk_std29', axis = 1, inplace = True)
    m_2.drop('err_type_weorwk_max29', axis = 1, inplace = True)
    return m, std, m_2

In [157]:
def wk(df,num_df_user,first_index):

#     df = df.loc[(df.datetime >= pd.to_datetime('2020-11-01 00:00:00')) & (df.datetime <= pd.to_datetime('2020-11-30 23:59:59'))]
    datas = df[['user_id','errtype','week']]
    df_ = datas[['user_id','week','errtype']].value_counts().to_frame().reset_index()
    df_ = df_.sort_values(['user_id','week']).rename(columns = {0:'counts'}).reset_index(drop = True)

    df1 = df_.loc[df_.week == 44][['user_id','errtype','counts']].values
    df2 = df_.loc[df_.week == 45][['user_id','errtype','counts']].values
    df3 = df_.loc[df_.week == 46][['user_id','errtype','counts']].values
    df4 = df_.loc[df_.week == 47][['user_id','errtype','counts']].values
    df5 = df_.loc[df_.week == 48][['user_id','errtype','counts']].values
    df6 = df_.loc[df_.week == 49][['user_id','errtype','counts']].values
    
    day_data = np.zeros((num_df_user, 42, 6))
    for i, dfa in enumerate([df1, df2, df3, df4, df5, df6]):
        for inx, val1, val2 in dfa:
            day_data[:, :, i][inx-first_index,val1 - 1] = val2

    m = day_data.mean(axis = 2)
    std = day_data.std(axis = 2)
    m_2 = day_data.max(axis = 2)
    
    m = pd.DataFrame(m, columns = ['err_type_wk_mean' + str(i) for i in range(1, 43)])
    std = pd.DataFrame(std, columns = ['err_type_wk_std' + str(i) for i in range(1, 43)])
    m_2 = pd.DataFrame(m_2, columns = ['err_type_wk_max' + str(i) for i in range(1, 43)])

    m.drop('err_type_wk_mean29', axis = 1, inplace = True)
    std.drop('err_type_wk_std29', axis = 1, inplace = True)
    m_2.drop('err_type_wk_max29', axis = 1, inplace = True)
    
    return m, std, m_2

In [158]:
def dy(df,num_df_user,first_index): 
    
#     df = df.loc[(df.datetime >=pd.to_datetime('2020-11-01 00:00:00'))& (df.datetime<=pd.to_datetime('2020-11-30 23:59:59'))]
    datas = df[['user_id','errtype','day']]
    df_=datas[['user_id','day','errtype']].value_counts().to_frame().reset_index()
    df_ =df_.sort_values(['user_id','day']).rename(columns = {0:'counts'}).reset_index(drop=True)


    day_data = np.zeros((num_df_user,42,30))
    for i in range(30):
        dfa = df_.loc[df_['day']==(i+1)][['user_id','errtype','counts']].values
        for inx , val1 ,val2 in dfa:
            day_data[:,:,i][inx-first_index,val1-1] = val2

    m=day_data.mean(axis=2)
    std=day_data.std(axis=2)       
    m_2=day_data.max(axis=2)
    
    m=pd.DataFrame(m,columns=['err_type_dy_mean'+str(i) for i in range(1,43)])
    std=pd.DataFrame(std,columns=['err_type_dy_std'+str(i) for i in range(1,43)])
    m_2=pd.DataFrame(m_2,columns=['err_type_dy_max'+str(i) for i in range(1,43)])

    m.drop('err_type_dy_mean29',axis=1,inplace=True)
    std.drop('err_type_dy_std29',axis=1,inplace=True)
    m_2.drop('err_type_dy_max29',axis=1,inplace=True)
    
    return m,std,m_2

In [159]:
def hr(df,num_df_user,first_index):     
    
#     df = df.loc[(df.datetime >=pd.to_datetime('2020-11-01 00:00:00'))& (df.datetime<=pd.to_datetime('2020-11-30 23:59:59'))]
    datas = df[['user_id','errtype','hour']]
    df_=datas[['user_id','hour','errtype']].value_counts().to_frame().reset_index()
    df_ =df_.sort_values(['user_id','hour']).rename(columns = {0:'counts'}).reset_index(drop=True)


    day_data = np.zeros((num_df_user,42,24))
    for i in range(24):
        dfa = df_.loc[df_['hour']==i][['user_id','errtype','counts']].values
        for inx , val1 ,val2 in dfa:
            day_data[:,:,i][inx-first_index,val1-1] = val2

    m=day_data.mean(axis=2)
    std=day_data.std(axis=2)       
    m_2=day_data.max(axis=2)
    
    m=pd.DataFrame(m,columns=['err_type_hr_mean'+str(i) for i in range(1,43)])
    std=pd.DataFrame(std,columns=['err_type_hr_std'+str(i) for i in range(1,43)])
    m_2=pd.DataFrame(m_2,columns=['err_type_hr_max'+str(i) for i in range(1,43)])

    m.drop('err_type_hr_mean29',axis=1,inplace=True)
    std.drop('err_type_hr_std29',axis=1,inplace=True)
    m_2.drop('err_type_hr_max29',axis=1,inplace=True)
    
    return m,std,m_2

In [160]:
# errcode 중복 정리

train_err['errcode'] = train_err['errcode'].apply(lambda x: str(x).strip())
test_err['errcode'] = test_err['errcode'].apply(lambda x: str(x).strip())

In [161]:
def make_int(row):
    if ',' in str(row):
        string = str(row).replace(',', '')
    elif '.' in str(row):
        string = str(row).replace('.', '')
    else:
        string = str(row)
    return int(string)

def make_col_prefix(dataframe):
    new_c = dataframe.columns
    new_cols = []
    for i in new_c:
        if str(i).isdigit() == True:
            newname = 'errtype_'+str(i)
            new_cols.append(newname)
        else:
            new_cols.append(i)
    return new_cols

def make_dataframe(err, qual, train=True):
    if train == True:
        err_users = err['user_id'].unique() # error data에 있는 unique 유저들
        df = pd.DataFrame() # 빈 데이터프레임 생성
        df['user_id'] = err_users # user 생성
    else:
        df = test_submission[['user_id']]
    
    err['count'] = 1 # count 세기 위한 용도로 컬럼 추가
    
    
    # 1) model_nm : model_nm를 pivot_table로 만듬
    print('making model_nm data...')
    err_model_nm = pd.pivot_table(err, index='user_id', columns='model_nm', values='count', fill_value=0)
    err_model_nm = err_model_nm.reset_index()
    print('done.')
    
    # 2) errtype : errtype을 pivot_table로 만듬
    print('making errtype data...')
    err_type = pd.pivot_table(err, index='user_id', columns='errtype', values='count', aggfunc='sum', fill_value=0)
    err_type = err_type.reset_index()
    err_type_cols = make_col_prefix(err_type)
    err_type.columns = err_type_cols
    print('done.')
    
    # 3) errcode : errcode 상위 50개를 pivot_table로 만듬.
    print('making errcode data...')
    errcode_50 = train_err['errcode'].value_counts(ascending=False)[:50].index # train_err에서 나온 상위 50개 errcode
    err_code = err[err['errcode'].isin(errcode_50)] # errcode_50에 해당하는 errcode만 뽑음.
    err_code = pd.pivot_table(err_code, index=['user_id'], columns='errcode', values='count', fill_value=0, aggfunc='sum').reset_index()
    print('done.')
    
    # 4) quality_issue : qual에 있는 사용자만을 뽑아서 만듬.
    print('making quality_issue data...')
    qual_id = qual.user_id.unique()
    print('done.')
     
    # 5) quality_0 ~ quality_12 : qual 에서 먼저 nan값 0처리해주고, 쉼표/점 처리하여 str -> int 화해야함.
    print('making quality_log data...')
    qual.fillna(0, inplace=True)
    for i in range(13):
        qual[f'quality_{i}'] = qual[f'quality_{i}'].apply(make_int)
    qual = qual.groupby(['user_id']).sum().reset_index()
    qual = qual.drop(columns='time')
    print('done.')
    
    # 6) weekend: 주말 =1, 주중 0
    print('making weekend data...')
    wk=pd.pivot_table(err, index=['user_id'], columns=['we_or_wk'], values='count', aggfunc='sum', fill_value=0).reset_index()
    wk_col=['weekend_'+str(i) for i in wk.columns[1:]]
    wk_col.insert(0,'user_id')
    wk.columns=wk_col
    print('done.')
    
    # df에 column 추가 : model_nm, errtype, errcode, quality_issue, quality_0~12, weekend, time 순으로
    print('Merging data into DataFrame...')
    df = pd.merge(df, err_model_nm, on=['user_id'], how='left')
    df = pd.merge(df, err_type, on=['user_id'], how='left')
    df = pd.merge(df, err_code, on=['user_id'], how='left')
    df['quality_issue'] = 0
    df.loc[df['user_id'].isin(qual_id), 'quality_issue'] = 1
    df = pd.merge(df, qual, on=['user_id'], how='left')
    df = pd.merge(df, wk, on=['user_id'], how='left')
    
    #전체 건수 합계 추가 
    df['total_err']=df.loc[:,['errtype_'+str(i) for i in range(1,29)]+['errtype_'+str(i) for i in range(30,43)]].sum(axis=1)
    
    # 새로 만든 파생변수들을 모두 merge 하면 됨
    if train == True:
        df = pd.merge(df, time_tr, on=['user_id'], how='left')
        df = pd.merge(df, fwver_tr, on=['user_id'], how='left')
    else:
        df = pd.merge(df, time_te, on=['user_id'], how='left')
        df = pd.merge(df, fwver_te, on=['user_id'], how='left')
        
    print('done.')
    
    df.fillna(0, inplace=True) # quality log 가 없는 유저들은 nan -> 0으로 처리해줘야함.
    
    if train == True: # train data인 경우에는 라벨링 값이 필요
        angry_users = train_problem['user_id'].unique()
        df['angry'] = [1 if user in angry_users else 0 for user in df['user_id']] # label data : 불만사용자면 1, 아니면 0 으로 라벨링
        
    return df

In [162]:
def err_merge(df,err,train):
# df에 column 추가 :주말여부별, 주별, 일별, 시간별 에러타입 변수
    print('Making_final_dataframe...')
    if train == True:
        
        m_wok,std_wok,m2_wok=we_or_wk(err,15000,10000)
        m_wk,std_wk,m2_wk=wk(err,15000,10000)
        m_dy,std_dy,m2_dy=dy(err,15000,10000)
        m_hr,std_hr,m2_hr=hr(err,15000,10000)
        
        df=pd.concat([df,m_wok,std_wok,m2_wok,m_wk,std_wk,m2_wk,m_dy,std_dy,m2_dy,m_hr,std_hr,m2_hr],axis=1)
    else:
        m_wok,std_wok,m2_wok=we_or_wk(err,14999,30000)
        m_wk,std_wk,m2_wk=wk(err,14999,30000)
        m_dy,std_dy,m2_dy=dy(err,14999,30000)
        m_hr,std_hr,m2_hr=hr(err,14999,30000)
        
        df=pd.concat([df,m_wok,std_wok,m2_wok,m_wk,std_wk,m2_wk,m_dy,std_dy,m2_dy,m_hr,std_hr,m2_hr],axis=1)

    print('done.')
    return df

In [163]:
train_df = make_dataframe(train_err, train_qual, True)

making model_nm data...
done.
making errtype data...
done.
making errcode data...
done.
making quality_issue data...
done.
making quality_log data...
done.
making weekend data...
done.
Merging data into DataFrame...
done.


In [164]:
train_df=err_merge(train_df,train_err,True)

Making_final_dataframe...
done.


In [165]:
test_df = make_dataframe(test_err, test_qual, False)

making model_nm data...
done.
making errtype data...
done.
making errcode data...
done.
making quality_issue data...
done.
making quality_log data...
done.
making weekend data...
done.
Merging data into DataFrame...
done.


In [166]:
test_df=err_merge(test_df,test_err,False)

Making_final_dataframe...
done.


In [167]:
train_df

,user_id,model_0,model_1,model_2,model_3,model_4,model_5,model_6,model_7,model_8,errtype_1,errtype_2,errtype_3,errtype_4,errtype_5,errtype_6,errtype_7,errtype_8,errtype_9,errtype_10,errtype_11,errtype_12,errtype_13,errtype_14,errtype_15,errtype_16,errtype_17,errtype_18,errtype_19,errtype_20,errtype_21,errtype_22,errtype_23,errtype_24,errtype_25,errtype_26,errtype_27,errtype_28,errtype_30,errtype_31,errtype_32,errtype_33,errtype_34,errtype_35,errtype_36,errtype_37,errtype_38,errtype_39,errtype_40,errtype_41,errtype_42,0,1,100,104,105,13,14,2,21,3,4,5,6,77,78,79,8.0,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,99,B-51049,B-A8002,H-51042,J-30021,NFANDROID2,Q-64002,S-61001,S-65002,UNKNOWN,V-21008,active,connection fail to establish,connection timeout,connectionterminated by local host,standby,terminate by peer user,quality_issue,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12,weekend_0,weekend_1,total_err,time_06_11,time_12_17,time_18_23,time_00_05,fwver,angry,err_type_weorwk_mean1,err_type_weorwk_mean2,err_type_weorwk_mean3,err_type_weorwk_mean4,err_type_weorwk_mean5,err_type_weorwk_mean6,err_type_weorwk_mean7,err_type_weorwk_mean8,err_type_weorwk_mean9,err_type_weorwk_mean10,err_type_weorwk_mean11,err_type_weorwk_mean12,err_type_weorwk_mean13,err_type_weorwk_mean14,err_type_weorwk_mean15,err_type_weorwk_mean16,err_type_weorwk_mean17,err_type_weorwk_mean18,err_type_weorwk_mean19,err_type_weorwk_mean20,err_type_weorwk_mean21,err_type_weorwk_mean22,err_type_weorwk_mean23,err_type_weorwk_mean24,err_type_weorwk_mean25,err_type_weorwk_mean26,err_type_weorwk_mean27,err_type_weorwk_mean28,err_type_weorwk_mean30,err_type_weorwk_mean31,err_type_weorwk_mean32,err_type_weorwk_mean33,err_type_weorwk_mean34,err_type_weorwk_mean35,err_type_weorwk_mean36,err_type_weorwk_mean37,err_type_weorwk_mean38,err_type_weorwk_mean39,err_type_weorwk_mean40,err_type_weorwk_mean41,err_type_weorwk_mean42,err_type_weorwk_std1,err_type_weorwk_std2,err_type_weorwk_std3,err_type_weorwk_std4,err_type_weorwk_std5,err_type_weorwk_std6,err_type_weorwk_std7,err_type_weorwk_std8,err_type_weorwk_std9,err_type_weorwk_std10,err_type_weorwk_std11,err_type_weorwk_std12,err_type_weorwk_std13,err_type_weorwk_std14,err_type_weorwk_std15,err_type_weorwk_std16,err_type_weorwk_std17,err_type_weorwk_std18,err_type_weorwk_std19,err_type_weorwk_std20,err_type_weorwk_std21,err_type_weorwk_std22,err_type_weorwk_std23,err_type_weorwk_std24,err_type_weorwk_std25,err_type_weorwk_std26,err_type_weorwk_std27,err_type_weorwk_std28,err_type_weorwk_std30,err_type_weorwk_std31,err_type_weorwk_std32,err_type_weorwk_std33,err_type_weorwk_std34,err_type_weorwk_std35,err_type_weorwk_std36,err_type_weorwk_std37,err_type_weorwk_std38,err_type_weorwk_std39,err_type_weorwk_std40,err_type_weorwk_std41,err_type_weorwk_std42,err_type_weorwk_max1,err_type_weorwk_max2,err_type_weorwk_max3,err_type_weorwk_max4,err_type_weorwk_max5,err_type_weorwk_max6,err_type_weorwk_max7,err_type_weorwk_max8,err_type_weorwk_max9,err_type_weorwk_max10,err_type_weorwk_max11,err_type_weorwk_max12,err_type_weorwk_max13,err_type_weorwk_max14,err_type_weorwk_max15,err_type_weorwk_max16,err_type_weorwk_max17,err_type_weorwk_max18,err_type_weorwk_max19,err_type_weorwk_max20,err_type_weorwk_max21,err_type_weorwk_max22,err_type_weorwk_max23,err_type_weorwk_max24,err_type_weorwk_max25,err_type_weorwk_max26,err_type_weorwk_max27,err_type_weorwk_max28,err_type_weorwk_max30,err_type_weorwk_max31,err_type_weorwk_max32,err_type_weorwk_max33,err_type_weorwk_max34,err_type_weorwk_max35,err_type_weorwk_max36,err_type_weorwk_max37,err_type_weorwk_max38,err_type_weorwk_max39,err_type_weorwk_max40,err_type_weorwk_max41,err_type_weorwk_max42,err_type_wk_mean1,err_type_wk_mean2,err_type_wk_mean3,err_type_wk_mean4,err_type_wk_mean5,err_type_wk_mean6,err_type_wk_mean7,err_type_wk_mean8,err_type_wk_mean9,err_type_wk_mean10,err_type_wk_mean11,err_type_wk_mean12,er

In [168]:
test_df

,user_id,model_0,model_1,model_2,model_3,model_4,model_5,model_6,model_7,model_8,errtype_1,errtype_2,errtype_3,errtype_4,errtype_5,errtype_6,errtype_7,errtype_8,errtype_9,errtype_10,errtype_11,errtype_12,errtype_13,errtype_14,errtype_15,errtype_16,errtype_17,errtype_18,errtype_19,errtype_20,errtype_21,errtype_22,errtype_23,errtype_24,errtype_25,errtype_26,errtype_27,errtype_28,errtype_30,errtype_31,errtype_32,errtype_33,errtype_34,errtype_35,errtype_36,errtype_37,errtype_38,errtype_39,errtype_40,errtype_41,errtype_42,0,1,100,104,105,13,14,2,21,3,4,5,6,77,78,79,8.0,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,99,B-51049,B-A8002,H-51042,J-30021,NFANDROID2,Q-64002,S-61001,S-65002,UNKNOWN,V-21008,active,connection fail to establish,connection timeout,connectionterminated by local host,standby,terminate by peer user,quality_issue,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12,weekend_0,weekend_1,total_err,time_06_11,time_12_17,time_18_23,time_00_05,fwver,err_type_weorwk_mean1,err_type_weorwk_mean2,err_type_weorwk_mean3,err_type_weorwk_mean4,err_type_weorwk_mean5,err_type_weorwk_mean6,err_type_weorwk_mean7,err_type_weorwk_mean8,err_type_weorwk_mean9,err_type_weorwk_mean10,err_type_weorwk_mean11,err_type_weorwk_mean12,err_type_weorwk_mean13,err_type_weorwk_mean14,err_type_weorwk_mean15,err_type_weorwk_mean16,err_type_weorwk_mean17,err_type_weorwk_mean18,err_type_weorwk_mean19,err_type_weorwk_mean20,err_type_weorwk_mean21,err_type_weorwk_mean22,err_type_weorwk_mean23,err_type_weorwk_mean24,err_type_weorwk_mean25,err_type_weorwk_mean26,err_type_weorwk_mean27,err_type_weorwk_mean28,err_type_weorwk_mean30,err_type_weorwk_mean31,err_type_weorwk_mean32,err_type_weorwk_mean33,err_type_weorwk_mean34,err_type_weorwk_mean35,err_type_weorwk_mean36,err_type_weorwk_mean37,err_type_weorwk_mean38,err_type_weorwk_mean39,err_type_weorwk_mean40,err_type_weorwk_mean41,err_type_weorwk_mean42,err_type_weorwk_std1,err_type_weorwk_std2,err_type_weorwk_std3,err_type_weorwk_std4,err_type_weorwk_std5,err_type_weorwk_std6,err_type_weorwk_std7,err_type_weorwk_std8,err_type_weorwk_std9,err_type_weorwk_std10,err_type_weorwk_std11,err_type_weorwk_std12,err_type_weorwk_std13,err_type_weorwk_std14,err_type_weorwk_std15,err_type_weorwk_std16,err_type_weorwk_std17,err_type_weorwk_std18,err_type_weorwk_std19,err_type_weorwk_std20,err_type_weorwk_std21,err_type_weorwk_std22,err_type_weorwk_std23,err_type_weorwk_std24,err_type_weorwk_std25,err_type_weorwk_std26,err_type_weorwk_std27,err_type_weorwk_std28,err_type_weorwk_std30,err_type_weorwk_std31,err_type_weorwk_std32,err_type_weorwk_std33,err_type_weorwk_std34,err_type_weorwk_std35,err_type_weorwk_std36,err_type_weorwk_std37,err_type_weorwk_std38,err_type_weorwk_std39,err_type_weorwk_std40,err_type_weorwk_std41,err_type_weorwk_std42,err_type_weorwk_max1,err_type_weorwk_max2,err_type_weorwk_max3,err_type_weorwk_max4,err_type_weorwk_max5,err_type_weorwk_max6,err_type_weorwk_max7,err_type_weorwk_max8,err_type_weorwk_max9,err_type_weorwk_max10,err_type_weorwk_max11,err_type_weorwk_max12,err_type_weorwk_max13,err_type_weorwk_max14,err_type_weorwk_max15,err_type_weorwk_max16,err_type_weorwk_max17,err_type_weorwk_max18,err_type_weorwk_max19,err_type_weorwk_max20,err_type_weorwk_max21,err_type_weorwk_max22,err_type_weorwk_max23,err_type_weorwk_max24,err_type_weorwk_max25,err_type_weorwk_max26,err_type_weorwk_max27,err_type_weorwk_max28,err_type_weorwk_max30,err_type_weorwk_max31,err_type_weorwk_max32,err_type_weorwk_max33,err_type_weorwk_max34,err_type_weorwk_max35,err_type_weorwk_max36,err_type_weorwk_max37,err_type_weorwk_max38,err_type_weorwk_max39,err_type_weorwk_max40,err_type_weorwk_max41,err_type_weorwk_max42,err_type_wk_mean1,err_type_wk_mean2,err_type_wk_mean3,err_type_wk_mean4,err_type_wk_mean5,err_type_wk_mean6,err_type_wk_mean7,err_type_wk_mean8,err_type_wk_mean9,err_type_wk_mean10,err_type_wk_mean11,err_type_wk_mean12,err_type

## Modeling 1_LightGBM

In [169]:
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
import lightgbm
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.metrics import roc_auc_score
# from bayes_opt import BayesianOptimization

In [170]:
import pandas as pd                         # 데이터 분석 라이브러리
import numpy as np                          # 계산 라이브러리
from tqdm import tqdm                       # 진행바
from sklearn.metrics import roc_auc_score   # AUC 스코어 계산
from sklearn.model_selection import KFold,StratifiedKFold   # K-fold CV    
from bayes_opt import BayesianOptimization  # 베이지안 최적화 라이브러리  
from functools import partial               # 함수 변수 고정
import lightgbm as lgb                      # LightGBM 라이브러리
import warnings                             
warnings.filterwarnings("ignore") 

In [171]:
train_x=train_df.drop(['user_id','angry'],axis=1)
test_x=test_df.iloc[:,1:]
y = train_df['angry']

In [172]:
def lgb_cv(num_leaves, learning_rate, n_estimators, subsample, colsample_bytree, reg_alpha, reg_lambda, x_data=None, y_data=None, n_splits=5, output='score'):
    score = 0
    kf = StratifiedKFold(n_splits=n_splits,random_state=2021,shuffle = True)
    models = []
    for train_index, valid_index in kf.split(x_data,y_data):
        x_train, y_train = x_data.iloc[train_index], y_data[train_index]
        x_valid, y_valid = x_data.iloc[valid_index], y_data[valid_index]
        
        model = lgb.LGBMClassifier(
            num_leaves = int(num_leaves), 
            learning_rate = learning_rate, 
            n_estimators = int(n_estimators), 
            subsample = np.clip(subsample, 0, 1), 
            colsample_bytree = np.clip(colsample_bytree, 0, 1), 
            reg_alpha = reg_alpha, 
            reg_lambda = reg_lambda
        )
        
        model.fit(x_train, y_train,eval_set=[(x_valid,y_valid)],early_stopping_rounds=300,eval_metric=['auc'],verbose=0)
        best_iter = model.best_iteration_
#         print(best_iter)
        models.append(model)
        
        pred = model.predict_proba(x_valid,num_iteration=best_iter)[:, 1]
        true = y_valid
        
        score += roc_auc_score(true, pred)/n_splits
    
    if output == 'score':
        return score
    if output == 'model':
        return models

### Bayesian_optimizing

In [ ]:
#  모델과 관련없는 변수 고정
func_fixed = partial(lgb_cv, x_data=train_x, y_data=y, n_splits=5, output='score') 
# 베이지안 최적화 범위 설정
lgbBO = BayesianOptimization(
    func_fixed, 
    {
        'num_leaves': (1, 500),        # num_leaves,       범위(16~1024)
        'learning_rate': (0.0001, 0.1),  # learning_rate,    범위(0.0001~0.1)
        'n_estimators': (1, 2000),      # n_estimators,     범위(16~1024)
        'subsample': (0.1, 1),             # subsample,        범위(0~1)
        'colsample_bytree': (0.1, 1),      # colsample_bytree, 범위(0~1)
        'reg_alpha': (0, 10),            # reg_alpha,        범위(0~10)
        'reg_lambda': (0, 50),           # reg_lambda,       범위(0~50)
    }, 
    random_state=4321                    # 시드 고정
)
lgbBO.maximize(init_points=15, n_iter=20) # 처음 5회 랜덤 값으로 score 계산 후 30회 최적화


In [ ]:
params = lgbBO.max['params']
params

In [ ]:
params = lgbBO.max['params']
models = lgb_cv(
    params['num_leaves'], 
    params['learning_rate'], 
    params['n_estimators'], 
    params['subsample'], 
    params['colsample_bytree'], 
    params['reg_alpha'], 
    params['reg_lambda'], 
    x_data=train_x, y_data=y, n_splits=10, output='model')

In [ ]:
# 모델 score
score

In [ ]:
preds = []
for model in models:
    pred = model.predict_proba(test_x)[:, 1]
    preds.append(pred)
pred = np.mean(preds, axis=0)

In [ ]:
test_submission['problem'] = pred
test_submission

## Modeling 2 _ Catboost

In [173]:
y=train_df['angry'].values
train_x=train_df.drop(['user_id','angry'],axis=1)
test_x=test_df.drop(['user_id'],axis=1)

In [174]:
def catboost_modeling(x_train, y_train, x_test, grow_policy, depth, learning_rate, l2_leaf_reg, random_seed,n_split):
    cat_model=[]
    aucs=[]

    test_pred = pd.Series([0 for x in range(len(x_test))], index=x_test.index)


    kf = StratifiedKFold(n_splits=n_split,random_state=2021,shuffle = True)
    for train_index, valid_index in kf.split(x_train,y_train):
        train_X, train_y = x_train.iloc[train_index], y_train[train_index]
        valid_X, valid_y = x_train.iloc[valid_index], y_train[valid_index]


        model = CatBoostClassifier(eval_metric = 'AUC',             
                                 iterations = 2000,               
                                 metric_period = 100,          
                                 early_stopping_rounds = 300,     
                                 task_type = 'CPU',                
                                 grow_policy = grow_policy,      
                                 depth = depth,                  
                                 learning_rate = learning_rate,  
                                 l2_leaf_reg = l2_leaf_reg,        
                                 random_seed = random_seed     
                                 
                                 )

        model.fit(train_X, train_y, eval_set=(valid_X, valid_y))
        aucs.append(model.best_score_['validation']['AUC'])
        cat_model.append(model)
        
    
        test_pred += model.predict_proba(x_test)[:,1] / (n_split)


    return test_pred,cat_model,aucs

In [ ]:
cat_result,models,aucs=catboost_modeling(train_x, y, test_x, 'Depthwise', 10, 0.03, 30, 2021,10)

In [ ]:
# k-fold 결과 평균 AUC
sum(aucs)/10

In [ ]:
# test data를 k-fold 모델들로 예측한 값의 평균
preds = []
for model in models:
    pred = model.predict_proba(test_x)[:, 1]
    preds.append(pred)
pred = np.mean(preds, axis=0)

In [ ]:
# 제출 결과물 저장
sample_submission = pd.read_csv('sample_submission.csv')
sample_submission['problem']=pred
sample_submission.to_csv('CatBoost_result.csv', index=False)